<a href="https://colab.research.google.com/github/SopheaEVC/water/blob/main/waterFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- The [dataRetrieval package in R](https://cran.r-project.org/web/packages/dataRetrieval/dataRetrieval.pdf) developed by [USGS](https://waterdata.usgs.gov/blog/moving-averages/).

In [ ]:
# code source: https://waterdata.usgs.gov/blog/moving-averages/
library(tidyverse)
library(dataRetrieval)

In [ ]:
# Retrieve daily Q
siteNumber <- c("01538000")
parameterCd <- "00060"
dailyQ <- readNWISdv(siteNumber, parameterCd)
dailyQ <- renameNWISColumns(dailyQ)
siteinfo <- readNWISsite(siteNumber)


### Calculate moving average

- The objective of this section is to calculate the moving average on all the flow data.



In [ ]:
# Checking for missing days
glimpse(dailyQ)

Rows: 38,200
Columns: 5
$ agency_cd <chr> "USGS", "USGS", "USGS", "USGS", "USGS", "USGS", "USGS", "USG…
$ site_no   <chr> "01538000", "01538000", "01538000", "01538000", "01538000", …
$ Date      <date> 1919-10-01, 1919-10-02, 1919-10-03, 1919-10-04, 1919-10-05,…
$ Flow      <dbl> 25, 25, 25, 25, 25, 30, 30, 30, 30, 30, 30, 35, 38, 44, 50, …
$ Flow_cd   <chr> "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", …


In [ ]:
range(dailyQ$Date)
diff(range(dailyQ$Date))
as.numeric(diff(range(dailyQ$Date))) # return a numeric
nrow(dailyQ)

[1] "1919-10-01" "2024-05-01"

Time difference of 38199 days

[1] 38199

[1] 38200

In [ ]:
if(as.numeric(diff(range(dailyQ$Date))) != (nrow(dailyQ)+1)){
    fullDates <- seq(
        from = min(dailyQ$Date),
        to = max(dailyQ$Date),
        by = '1 day'
    )
    fullDates <- data.frame(
        Date = fullDates,
        agency_cd = dailyQ$agency_cd[1],
        site_no = dailyQ$site_no,
        stringsAsFactors = FALSE
    )
    dailyQ <- full_join(
        dailyQ,
        fullDates,
        by = c("Date", "agency_cd", "site_no")
    ) %>%
      arrange(Date)
}
glimpse(dailyQ)

Rows: 38,200
Columns: 5
$ agency_cd <chr> "USGS", "USGS", "USGS", "USGS", "USGS", "USGS", "USGS", "USG…
$ site_no   <chr> "01538000", "01538000", "01538000", "01538000", "01538000", …
$ Date      <date> 1919-10-01, 1919-10-02, 1919-10-03, 1919-10-04, 1919-10-05,…
$ Flow      <dbl> 25, 25, 25, 25, 25, 30, 30, 30, 30, 30, 30, 35, 38, 44, 50, …
$ Flow_cd   <chr> "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", …


- To calculate a **rolling average**, add up all the data in a set and divide the total by the time period. For example, to calculate a **30-day rolling average**, add up the data from the past **30 days and divide by 30**.

In [ ]:
ma <- function(x,n=30){stats::filter(x,rep(1/n,n), sides=1)} #repeat 1/n for 30 times
dailyQ <- dailyQ %>%
  mutate(
  rollMean = as.numeric(ma(Flow)),
  day.of.year = as.numeric(strftime(
    Date,
    format = '%j'
  ))
  )
glimpse(dailyQ)



Rows: 38,200
Columns: 7
$ agency_cd   <chr> "USGS", "USGS", "USGS", "USGS", "USGS", "USGS", "USGS", "U…
$ site_no     <chr> "01538000", "01538000", "01538000", "01538000", "01538000"…
$ Date        <date> 1919-10-01, 1919-10-02, 1919-10-03, 1919-10-04, 1919-10-0…
$ Flow        <dbl> 25, 25, 25, 25, 25, 30, 30, 30, 30, 30, 30, 35, 38, 44, 50…
$ Flow_cd     <chr> "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A"…
$ rollMean    <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ day.of.year <dbl> 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285…


### Calculate historical percentiles

In [ ]:
summaryQ <- dailyQ %>%
  group_by(day.of.year) %>%
  summarize(
  p75 = quantile(rollMean, probs = .75, na.rm = TRUE),
  p25 = quantile(rollMean, probs = .25, na.rm = TRUE),
  p10 = quantile(rollMean, probs = .1, na.rm = TRUE),
  p05 = quantile(rollMean, probs = .05, na.rm = TRUE),
  p00 = quantile(rollMean, probs = 0, na.rm = TRUE)
  )

current.year <- as.numeric(strftime(Sys.Date(), format = "%y")) # time conversion
summary.0 <- summaryQ %>%
  mutate(
  Date = as.Date(day.of.year - 1, origin = paste0(current.year, '-01-01')),
  day.of.year = day.of.year + 365
  )
summary.1 <- summaryQ %>%
  mutate(
  Date = as.Date(
    day.of.year -1 ,
    origin = paste0(current.year, '-01-01')
  ),
  day.of.year = day.of.year + 365
  )
